# 패키지 임포트

In [6]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sys
sys.path.append('C:/Users/User/Desktop/20Newsgroups')
from data_preprocessing import *

# 데이터 로드

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [ ]:
textcuboid=np.load('./1-Channel textcuboid_20ng(elmo).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(elmo).npy')

# 데이터 전처리

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [7]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [8]:
one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [ ]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

# 모델 빌드

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [10]:
input_layer = Input(shape=(952, 256))
conv1 = Conv1D(256, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 952, 256)]        0         
                                                                 
 conv1d (Conv1D)             (None, 952, 256)          65792     
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                5140      
                                                             

# 모델 학습

In [15]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/50
55/55 [==============================] - 6s 89ms/step - loss: 2.3059 - accuracy: 0.3348 - val_loss: 1.1695 - val_accuracy: 0.6580 - lr: 0.0010
Epoch 2/50
55/55 [==============================] - 4s 73ms/step - loss: 1.1341 - accuracy: 0.6348 - val_loss: 0.8475 - val_accuracy: 0.7350 - lr: 0.0010
Epoch 3/50
55/55 [==============================] - 4s 71ms/step - loss: 0.8821 - accuracy: 0.7148 - val_loss: 0.7388 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 4/50
55/55 [==============================] - 4s 70ms/step - loss: 0.7524 - accuracy: 0.7549 - val_loss: 0.6983 - val_accuracy: 0.7870 - lr: 0.0010
Epoch 5/50
55/55 [==============================] - 4s 72ms/step - loss: 0.6785 - accuracy: 0.7834 - val_loss: 0.6559 - val_accuracy: 0.7900 - lr: 0.0010
Epoch 6/50
55/55 [==============================] - 4s 70ms/step - loss: 0.6094 - accuracy: 0.8030 - val_loss: 0.6501 - val_accuracy: 0.7910 - lr: 0.0010
Epoch 7/50
55/55 [==============================] - 4s 70ms/step - loss: 0.5

In [16]:
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 9ms/step - loss: 0.5730 - accuracy: 0.8433


[0.5730412006378174, 0.8433350920677185]

In [24]:
from keras.models import load_model
model = load_model('best_model_50.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 10ms/step - loss: 0.5441 - accuracy: 0.8473


[0.5441328883171082, 0.8473181128501892]